* **1bp** Update the cross validation example to accept MLP/CNN models, sending the configurations tested by a *Python dict*, with specific parameters to the model analyzed.

In [8]:
import numpy as np
import tensorflow.keras.datasets as datasets
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Flatten, Dropout,InputLayer, Conv2D, MaxPooling2D, Dense


def create_model(layers, optimizer, loss):
  model = Sequential()
  model.add(InputLayer(input_shape=(28, 28, 1)))
  for layer in layers:
    model.add(layer)

  model.add(Dense(10, activation='softmax')) # the definition of the classifier stage will be the same
  model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])
  return model

In [9]:
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# firslty, we will load the MNIST dataset
def get_mnist_data():
  (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
  X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
  X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
  Y_train = to_categorical(Y_train) # use one-hot encoding
  Y_test = to_categorical(Y_test)
  # we will experiment with data normalization later
  return X_train, Y_train, X_test, Y_test

X_train, Y_train, X_test, Y_test = get_mnist_data()

In [7]:
optimizers = [
  "adam",
  "sgd"
]

losses = [
    "categorical_crossentropy",
    "mse",
]



conv = [
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2))
]

dense = [
    Dense(64, activation='relu', kernel_initializer='he_uniform'),
]

best_model = None
best_acc = 0

for i in range(2):
    for j in range(1):
        layers = conv * i + [Flatten()] + dense * j
        for optimizer in optimizers:
            for loss in losses:
                model = create_model(layers, optimizer, loss)
                history = model.fit(X_train, Y_train, epochs=10, validation_split=0.2, verbose=0)
                test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=0)
                rez = test_acc
                if rez > best_acc:
                    best_acc = rez
                    best_model = model

print(best_acc, best_model)


0.9726999998092651 <keras.engine.sequential.Sequential object at 0x000001D925B99190>


In [8]:
best_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 2704)              0         
                                                                 
 dense_14 (Dense)            (None, 10)                27050     
                                                                 
Total params: 27,210
Trainable params: 27,210
Non-trainable params: 0
_________________________________________________________________


* **1bp** Implement the *K-Fold* cross-validation to evaluate your CNN model over the MNIST dataset. Keep the test data as your benchmark, using just the training data for architecture decisions. Compute the number of parameters, and compare to the result provided by *Keras*.

In [10]:
optimizers = [
  "adam",
  "sgd"
]

losses = [
    "categorical_crossentropy",
    "mse",
]



conv = [
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2))
]

dense = [
    Dense(64, activation='relu', kernel_initializer='he_uniform'),
]


In [16]:
best_model = None
best_acc = 0

for i in range(2):
    for j in range(1):
        layers = conv * i + [Flatten()] + dense * j
        for optimizer in optimizers:
            for loss in losses:
                x_train = np.split(X_train, 5)
                y_train = np.split(Y_train, 5)

                rez = 0
                for k_fold in range(5):
                    x_first  = None
                    y_first  = None
                    for k in range(5):
                        if k == k_fold:
                            continue
                        if y_first is None:
                            x_first = x_train[k]
                            y_first = y_train[k]
                        else:
                            x_first = np.concatenate((x_first, x_train[k]), axis=0)
                            y_first = np.concatenate((y_first, y_train[k]), axis=0)
                    x_val = x_train[k_fold]
                    y_val = y_train[k_fold]

                    model = create_model(layers, optimizer, loss)
                    history = model.fit(x_first, y_first, epochs=10, verbose=0)
                    val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
                    rez += val_acc

                rez = rez / 5
                if rez > best_acc:
                    best_acc = rez
                    best_model = model

print(best_acc, best_model)

0.9712000012397766 <keras.engine.sequential.Sequential object at 0x0000016B1367D220>


In [17]:
best_model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 2704)              0         
                                                                 
 dense_27 (Dense)            (None, 10)                27050     
                                                                 
Total params: 27,210
Trainable params: 27,210
Non-trainable params: 0
_________________________________________________________________


* **1bp** Update the CNN autoencoder to have a latent with the same size as the Linear AE designed last week (37x1), using a sequence of Dense layers on the flatenned filter output. Compare its performance to the Linear AE and the SVC classifier.

In [64]:

from keras import Input, Model
from keras.layers import Conv2D, MaxPooling2D, Reshape, UpSampling2D, Flatten


input_img = Input(shape=(28, 28, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x) # use pooling to reduce by half the width/height of the representation
encoded = Flatten()(x)

latent = Dense(units = 37, activation = 'relu')(encoded) #

encoded = Reshape((7, 7, 32))(encoded)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)

x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# compile the models for encoder and autoencoder
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

encoder = Model(input_img, latent)

In [65]:
X_train, Y_train, X_test, Y_test = get_mnist_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [67]:
autoencoder.fit(X_train, X_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(X_test, X_test))

Epoch 1/10
469/469 [==============================] - 35s 74ms/step - loss: 0.0748 - val_loss: 0.0729

In [68]:
X_enc_train = encoder.predict(X_train)
print(X_enc_train.shape)
X_enc_test = encoder.predict(X_test)

(60000, 37)


In [69]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


(_, Y_train), (_, Y_test) = mnist.load_data()

scaler = StandardScaler()
X_enc_train = scaler.fit_transform(X_enc_train)
X_enc_test = scaler.transform(X_enc_test)

clf =  SVC(C=1.5, kernel='rbf', gamma='auto')
clf.fit(X_enc_train, Y_train)
predictions = clf.predict(X_enc_test)


In [70]:
print("Classification accuracy: {}".format(accuracy_score(Y_test, predictions)))

Classification accuracy: 0.8908


The classification accuracy is lower, but it might be due to training only 10 epochs and not 30.


* **1bp** Using strided convolution, implement a binary classifier for two of the MNIST classes, using *Dense* layers only where it is exclusively necessary (reduce the dimensions of the output to 1x1 output using strides).

In [ ]:
X_train, Y_train, X_test, Y_test = get_mnist_data()

x_first = None
y_first = None


for i in range(X_train.shape[0]):
    x = np.argmax(Y_train[i])
    if x > 1:
        continue
    if x_first is None:
        x_first = np.array([X_train[i]])
        y_first = np.array([x])
    else:
        x_first = np.concatenate((x_first, [X_train[i]]), axis=0)
        y_first = np.concatenate((y_first, [x]), axis=0)

x_test = None
y_test = None

for i in range(X_test.shape[0]):
    x = np.argmax(Y_test[i])
    if x > 1:
        continue
    if x_test is None:
        x_test = np.array([X_test[i]])
        y_test = np.array([x])
    else:
        x_test = np.concatenate((x_test, [X_test[i]]), axis=0)
        y_test = np.concatenate((y_test, [x]), axis=0)


In [74]:
model = Sequential()
model.add(InputLayer(input_shape=(28, 28, 1)))
model.add(Conv2D(16, (3, 3), strides=(2, 2), activation='relu')) # 14 x 14
# model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), strides=(2, 2), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), strides=(2, 2), activation='relu'))
model.add(Conv2D(1, (2, 2), activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_88 (Conv2D)          (None, 13, 13, 16)        160       
                                                                 
 conv2d_89 (Conv2D)          (None, 6, 6, 16)          2320      
                                                                 
 conv2d_90 (Conv2D)          (None, 2, 2, 16)          2320      
                                                                 
 conv2d_91 (Conv2D)          (None, 1, 1, 1)           65        
                                                                 
 flatten_18 (Flatten)        (None, 1)                 0         
                                                                 
 dense_52 (Dense)            (None, 1)                 2         
                                                                 
Total params: 4,867
Trainable params: 4,867
Non-train

In [ ]:
print(X_train.shape)
print(y_first)

model.fit(x_first, y_first, epochs=10)

In [ ]:
model.evaluate(x_test, y_test)